In [65]:
import Pkg; Pkg.status()
include("../src/Arthur.jl")
using RigidBodyDynamics
using StaticArrays
using RobotDynamics
using Rotations
using LinearAlgebra
using ForwardDiff, FiniteDiff
using Altro
using TrajectoryOptimization

      Status `~/catkin_workspace/src/ros_kortex/kortex_control/mpc/notebooks/Project.toml` (empty project)


In [66]:
model = Arthur()
print(typeof((dynamics(model, zeros(32), 0.1*ones(7)))))
n,m = size(model)

Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
SVector{32, Float64}

(32, 7)

In [67]:
N = 61
tf = 3.
dt = tf/(N-1)

0.05

In [68]:
# TODO:
# Uref could be just U equilibrium (just gravity compensation),
# Xref could be the desired joint and cartesian positions and velocities, and then 0 for the forces
Uref = [fill(0.0, m) for k = 1:N-1]
Xref = [fill(0.0, n) for k = 1:N]
dtref = [dt for k=1:N]
traj = RobotDynamics.Traj(Xref, Uref, dtref, cumsum(dtref) .- dtref[1]);

In [69]:
# Set up
# TODO: Tune weights
Q = 1.0e-2*Diagonal(@SVector ones(n))
Qf = 100.0*Diagonal(@SVector ones(n))
R = 1.0e-1*Diagonal(@SVector ones(m))
obj = TrajectoryOptimization.TrackingObjective(Q, R, traj, Qf=Qf)
# obj = LQRObjective(Q,R,Qf,xf,N);

In [70]:
# Create Empty ConstraintList
conSet = ConstraintList(n,m,N)

# Control Bounds based on Robot Specs (Joint torque limits)
u_bnd = [39.0, 39.0, 39.0, 39.0, 9.0, 9.0, 9.0]
control_bnd = BoundConstraint(n,m, u_min=-u_bnd, u_max=u_bnd)
add_constraint!(conSet, control_bnd, 1:N-1)

# State Bounds based on Robot Specs (Joint velocity and speed limits)
x_bnd = zeros(n)
x_bnd[1:7] = [Inf, deg2rad(128.9), Inf, deg2rad(147.8), Inf, deg2rad(120.3), Inf] # rad
x_bnd[8:14] = [1.39, 1.39, 1.39, 1.39, 1.22, 1.22, 1.22] # rad/sec
x_bnd[15:end] = [Inf for k=1:(n-14)] # Constraints on force elsewhere
state_bnd = BoundConstraint(n,m, x_min=-x_bnd, x_max=x_bnd)
add_constraint!(conSet, state_bnd, 1:N)

# Cartesian Velocity Bound
ẋ_max = 0.0005 # m/s
vel_bnd = NormConstraint(n, m, ẋ_max, Inequality(), 21:23)
add_constraint!(conSet, vel_bnd, 1:N)

# Force Bound (Fx Fy Fz)
F_max = 20 # Newtons
F_bnd = NormConstraint(n, m, F_max, Inequality(), 27:29)
add_constraint!(conSet, F_bnd, 1:N)

# Other possible constraints:
# Cartesian angular velocity bounds?
# External moment bounds?

# Goal Constraint - only if you want the final state to be the desired state
# goal = GoalConstraint(xf)
# add_constraint!(conSet, goal, N)

In [71]:
prob = Problem(model, obj, Xref[end], tf, x0=Xref[1], constraints=conSet, N=N, U0=Uref, X0=Xref)

Problem{RK3, Float64}(Arthur{Any}(Spanning tree:
Vertex: world (root)
  Vertex: base_link, Edge: base_link_to_world
    Vertex: Shoulder_Link, Edge: Actuator1
      Vertex: HalfArm1_Link, Edge: Actuator2
        Vertex: HalfArm2_Link, Edge: Actuator3
          Vertex: ForeArm_Link, Edge: Actuator4
            Vertex: SphericalWrist1_Link, Edge: Actuator5
              Vertex: SphericalWrist2_Link, Edge: Actuator6
                Vertex: Bracelet_Link, Edge: Actuator7
                  Vertex: EndEffector_Link, Edge: EndEffector
No non-tree joints.), Objective, ConstraintList(32, 7, TrajectoryOptimization.AbstractConstraint[BoundConstraint{14, 39, Float64}(32, 7, [Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf  …  Inf, Inf, Inf, 39.0, 39.0, 39.0, 39.0, 9.0, 9.0, 9.0], [-Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf  …  -Inf, -Inf, -Inf, -39.0, -39.0, -39.0, -39.0, -9.0, -9.0, -9.0], [449, 464, 479, 494, 509, 524, 539], [456, 471, 486, 501, 516, 531, 546], [33, 34, 35, 36, 

In [72]:
using Altro
opts = SolverOptions(
    cost_tolerance_intermediate=1e-2,
    penalty_scaling=10.,
    penalty_initial=1.0
)

altro = ALTROSolver(prob, opts)
set_options!(altro, show_summary=true)
solve!(altro);

Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
1Check2.2
1Check2.3
1Check2.4
1Check2.5
1Check2.6
2Check2.2
2Check2.3
2Check2.4
2Check2.5
2Check2.6
3Check2.2
3Check2.3
3Check2.4
3Check2.5
3Check2.6
4Check2.2
4Check2.3
4Check2.4
4Check2.5
4Check2.6
5Check2.2
5Check2.3
5Check2.4
5Check2.5
5Check2.6
6Check2.2
6Check2.3
6Check2.4
6Check2.5
6Check2.6
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
1Check2.2
1Check2.3
1Check2.4
1Check2.5
1Check2.6
2Check2.2
2Check2.3
2Check2.4
2Check2.5
2Check2.6
3Check2.2
3Check2.3
3Check2.4
3Check2.5
3Check2.6
4Check2.2
4Check2.3
4Check2.4
4Check2.5
4Check2.6
5Check2.2
5Check2.3
5Check2.4
5Check2.5
5Check2.6
6Check2.2
6Check2.3
6Check2.4
6Check2.5
6Check2.6
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
1Check2.2
1Check2.3
1Check2.4
1Check2.5
1Check2.6
2Check2.2
2Check2.3
2Check2.4
2Check2.5
2Check2.6
3Check2.2
3Check2.3
3Check2.

4Check2.5
4Check2.6
5Check2.2
5Check2.3
5Check2.4
5Check2.5
5Check2.6
6Check2.2
6Check2.3
6Check2.4
6Check2.5
6Check2.6
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
1Check2.2
1Check2.3
1Check2.4
1Check2.5
1Check2.6
2Check2.2
2Check2.3
2Check2.4
2Check2.5
2Check2.6
3Check2.2
3Check2.3
3Check2.4
3Check2.5
3Check2.6
4Check2.2
4Check2.3
4Check2.4
4Check2.5
4Check2.6
5Check2.2
5Check2.3
5Check2.4
5Check2.5
5Check2.6
6Check2.2
6Check2.3
6Check2.4
6Check2.5
6Check2.6
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
1Check2.2
1Check2.3
1Check2.4
1Check2.5
1Check2.6
2Check2.2
2Check2.3
2Check2.4
2Check2.5
2Check2.6
3Check2.2
3Check2.3
3Check2.4
3Check2.5
3Check2.6
4Check2.2
4Check2.3
4Check2.4
4Check2.5
4Check2.6
5Check2.2
5Check2.3
5Check2.4
5Check2.5
5Check2.6
6Check2.2
6Check2.3
6Check2.4
6Check2.5
6Check2.6
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
1Check2.2
1Check2.3
1Check2.4
1

2Check2.5
2Check2.6
3Check2.2
3Check2.3
3Check2.4
3Check2.5
3Check2.6
4Check2.2
4Check2.3
4Check2.4
4Check2.5
4Check2.6
5Check2.2
5Check2.3
5Check2.4
5Check2.5
5Check2.6
6Check2.2
6Check2.3
6Check2.4
6Check2.5
6Check2.6
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
1Check2.2
1Check2.3
1Check2.4
1Check2.5
1Check2.6
2Check2.2
2Check2.3
2Check2.4
2Check2.5
2Check2.6
3Check2.2
3Check2.3
3Check2.4
3Check2.5
3Check2.6
4Check2.2
4Check2.3
4Check2.4
4Check2.5
4Check2.6
5Check2.2
5Check2.3
5Check2.4
5Check2.5
5Check2.6
6Check2.2
6Check2.3
6Check2.4
6Check2.5
6Check2.6
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
1Check2.2
1Check2.3
1Check2.4
1Check2.5
1Check2.6
2Check2.2
2Check2.3
2Check2.4
2Check2.5
2Check2.6
3Check2.2
3Check2.3
3Check2.4
3Check2.5
3Check2.6
4Check2.2
4Check2.3
4Check2.4
4Check2.5
4Check2.6
5Check2.2
5Check2.3
5Check2.4
5Check2.5
5Check2.6
6Check2.2
6Check2.3
6Check2.4
6Check2.5
6Check2.6
Check3
Check4


Check7
Check8
Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
1Check2.2
1Check2.3
1Check2.4
1Check2.5
1Check2.6
2Check2.2
2Check2.3
2Check2.4
2Check2.5
2Check2.6
3Check2.2
3Check2.3
3Check2.4
3Check2.5
3Check2.6
4Check2.2
4Check2.3
4Check2.4
4Check2.5
4Check2.6
5Check2.2
5Check2.3
5Check2.4
5Check2.5
5Check2.6
6Check2.2
6Check2.3
6Check2.4
6Check2.5
6Check2.6
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
1Check2.2
1Check2.3
1Check2.4
1Check2.5
1Check2.6
2Check2.2
2Check2.3
2Check2.4
2Check2.5
2Check2.6
3Check2.2
3Check2.3
3Check2.4
3Check2.5
3Check2.6
4Check2.2
4Check2.3
4Check2.4
4Check2.5
4Check2.6
5Check2.2
5Check2.3
5Check2.4
5Check2.5
5Check2.6
6Check2.2
6Check2.3
6Check2.4
6Check2.5
6Check2.6
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
1Check2.2
1Check2.3
1Check2.4
1

Check1
Check2
Check2.1
1Check2.2
1Check2.3
1Check2.4
1Check2.5
1Check2.6
2Check2.2
2Check2.3
2Check2.4
2Check2.5
2Check2.6
3Check2.2
3Check2.3
3Check2.4
3Check2.5
3Check2.6
4Check2.2
4Check2.3
4Check2.4
4Check2.5
4Check2.6
5Check2.2
5Check2.3
5Check2.4
5Check2.5
5Check2.6
6Check2.2
6Check2.3
6Check2.4
6Check2.5
6Check2.6
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
1Check2.2
1Check2.3
1Check2.4
1Check2.5
1Check2.6
2Check2.2
2Check2.3
2Check2.4
2Check2.5
2Check2.6
3Check2.2
3Check2.3
3Check2.4
3Check2.5
3Check2.6
4Check2.2
4Check2.3
4Check2.4
4Check2.5
4Check2.6
5Check2.2
5Check2.3
5Check2.4
5Check2.5
5Check2.6
6Check2.2
6Check2.3
6Check2.4
6Check2.5
6Check2.6
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
1Check2.2
1Check2.3
1Check2.4
1Check2.5
1Check2.6
2Check2.2
2Check2.3
2Check2.4
2Check2.5
2Check2.6
3Check2.2
3Check2.3
3Check2.4
3Check2.5
3Check2.6
4Check2.2
4Check2.3
4Check2.4
4Check2.5
4Check2.6
5Check2.2
5

...

6Check2.2
6Check2.3
6Check2.4
6Check2.5
6Check2.6
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
1Check2.2
1Check2.3
1Check2.4
1Check2.5
1Check2.6
2Check2.2
2Check2.3
2Check2.4
2Check2.5
2Check2.6
3Check2.2
3Check2.3
3Check2.4
3Check2.5
3Check2.6
4Check2.2
4Check2.3
4Check2.4
4Check2.5
4Check2.6
5Check2.2
5Check2.3
5Check2.4
5Check2.5
5Check2.6
6Check2.2
6Check2.3
6Check2.4
6Check2.5
6Check2.6
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
1Check2.2
1Check2.3
1Check2.4
1Check2.5
1Check2.6
2Check2.2
2Check2.3
2Check2.4
2Check2.5
2Check2.6
3Check2.2
3Check2.3
3Check2.4
3Check2.5
3Check2.6
4Check2.2
4Check2.3
4Check2.4
4Check2.5
4Check2.6
5Check2.2
5Check2.3
5Check2.4
5Check2.5
5Check2.6
6Check2.2
6Check2.3
6Check2.4
6Check2.5
6Check2.6
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
1Check2.2
1Check2.3
1Check2.4
1Check2.5
1Check2.6
2Check2.2
2Check2.3
2Check2.4
2Check2.5
2Check2.6
3

Check7
Check8
Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Chec

4Check2.5
4Check2.6
5Check2.2
5Check2.3
5Check2.4
5Check2.5
5Check2.6
6Check2.2
6Check2.3
6Check2.4
6Check2.5
6Check2.6
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
1Check2.2
1Check2.3
1Check2.4
1Check2.5
1Check2.6
2Check2.2
2Check2.3
2Check2.4
2Check2.5
2Check2.6
3Check2.2
3Check2.3
3Check2.4
3Check2.5
3Check2.6
4Check2.2
4Check2.3
4Check2.4
4Check2.5
4Check2.6
5Check2.2
5Check2.3
5Check2.4
5Check2.5
5Check2.6
6Check2.2
6Check2.3
6Check2.4
6Check2.5
6Check2.6
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
1Check2.2
1Check2.3
1Check2.4
1Check2.5
1Check2.6
2Check2.2
2Check2.3
2Check2.4
2Check2.5
2Check2.6
3Check2.2
3Check2.3
3Check2.4
3Check2.5
3Check2.6
4Check2.2
4Check2.3
4Check2.4
4Check2.5
4Check2.6
5Check2.2
5Check2.3
5Check2.4
5Check2.5
5Check2.6
6Check2.2
6Check2.3
6Check2.4
6Check2.5
6Check2.6
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
1Check2.2
1Check2.3
1Check2.4
1

2Check2.5
2Check2.6
3Check2.2
3Check2.3
3Check2.4
3Check2.5
3Check2.6
4Check2.2
4Check2.3
4Check2.4
4Check2.5
4Check2.6
5Check2.2
5Check2.3
5Check2.4
5Check2.5
5Check2.6
6Check2.2
6Check2.3
6Check2.4
6Check2.5
6Check2.6
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
1Check2.2
1Check2.3
1Check2.4
1Check2.5
1Check2.6
2Check2.2
2Check2.3
2Check2.4
2Check2.5
2Check2.6
3Check2.2
3Check2.3
3Check2.4
3Check2.5
3Check2.6
4Check2.2
4Check2.3
4Check2.4
4Check2.5
4Check2.6
5Check2.2
5Check2.3
5Check2.4
5Check2.5
5Check2.6
6Check2.2
6Check2.3
6Check2.4
6Check2.5
6Check2.6
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
1Check2.2
1Check2.3
1Check2.4
1Check2.5
1Check2.6
2Check2.2
2Check2.3
2Check2.4
2Check2.5
2Check2.6
3Check2.2
3Check2.3
3Check2.4
3Check2.5
3Check2.6
4Check2.2
4Check2.3
4Check2.4
4Check2.5
4Check2.6
5Check2.2
5Check2.3
5Check2.4
5Check2.5
5Check2.6
6Check2.2
6Check2.3
6Check2.4
6Check2.5
6Check2.6
Check3
Check4


Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Check2
Check2.1
Check3
Check4
Check5
Check5.1
Check5.2
Check6
Check7
Check8
Check1
Chec